In [2]:
%load_ext autoreload
%autoreload 2

import numpy as np
import json
from copy import deepcopy

In [ ]:
from camera_conventions.geometry import Geometry

arr = np.array([    
    0.0, 0.2, 0.5,
    0.2, -0.1, 0.3,
    0.2, -0.1, 0.7,
    0.0, 0.2, 0.5, 
    -0.2, -0.1, 0.7,
    0.2, -0.1, 0.7,
    0.0, 0.2, 0.5, 
    -0.2, -0.1, 0.3,
    -0.2, -0.1, 0.7,
    0.0, 0.2, 0.5,      
    -0.2, -0.1, 0.3,
    0.2, -0.1, 0.3,
    0.2, -0.1, 0.3, 
    0.2, -0.1, 0.7,
    -0.2, -0.1, 0.7,
    -0.2, -0.1, 0.3, 
    0.2, -0.1, 0.3,
    -0.2, -0.1, 0.7]).reshape(-1, 3)
colors = np.array([
    1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0,
                  0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0,
                  0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0,
                  1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,
                  1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0
]).reshape(-1, 3)
arr[:, 2] -= 0.5
verts = arr
faces = np.arange(0, verts.shape[0]).reshape(-1, 3)
geom = Geometry(convention="OpenGL", verts=verts, faces=faces, vertex_colors=colors)

In [ ]:
geom = Geometry(convention="OpenGL", verts=verts, faces=faces)
geom.convert("Blender").dump('../assets/geometries/pyramid_blender.json')

In [14]:
C = np.array([1., 0, 1])
# look at the origin
cameraZ = np.array([1., 0, 1])
cameraZ /= np.linalg.norm(cameraZ)
cameraY = np.array([0., 1, 0])
cameraY = cameraY - np.dot(cameraY, cameraZ) * cameraZ
cameraY /= np.linalg.norm(cameraY)
assert np.isclose(np.dot(cameraY, cameraZ), 0)
cameraX = np.cross(cameraY, cameraZ)
cameraX /= np.linalg.norm(cameraX)
assert np.isclose(np.dot(cameraX, cameraY), 0)
assert np.isclose(np.dot(cameraX, cameraZ), 0)

# cameraX, cameraY, cameraZ are the columns of the camera-to-world matrix
# we want the world-to-camera matrix
cameraToWorld = np.array([cameraX, cameraY, cameraZ]).T
R = np.linalg.inv(cameraToWorld)
T = -R @ C
viewMatrix = np.eye(4)
viewMatrix[:3, :3] = R
viewMatrix[:3, 3] = T
viewMatrix

array([[ 0.70710678,  0.        , -0.70710678,  0.        ],
       [ 0.        ,  1.        ,  0.        ,  0.        ],
       [ 0.70710678,  0.        ,  0.70710678, -1.41421356],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [15]:
np.allclose(C, viewMatrix[:3, :3].T @ -viewMatrix[:3, 3])

True

In [18]:
from camera_conventions.camera import Camera
camTest = Camera(
    fx=100, fy=100, cx=50, cy=50, width=100, height=100, # these are not implemented yet
    T=viewMatrix, convention="OpenGL")
camTest.to_txt('../assets/cameras/testCam.txt')
camTest.to_json('../assets/cameras/testCam.json')
camTest.convert("Blender").to_json('../assets/cameras/testCam_blender.json')

In [ ]:
b_cam = camTest.convert("Blender")
b_cam.T = np.linalg.inv(b_cam.T)
b_cam.to_json('../assets/cameras/testCam_blender.json')
b_cam